In [28]:
# ! Taken from https://github.com/locuslab/DC3/blob/main/datasets/simple/make_dataset.py

import numpy as np
import pickle
import torch
import numpy as np
import osqp
from qpth.qp import QPFunction
from scipy.sparse import csc_matrix

import time
import os

torch.set_default_dtype(torch.float64)
from QP_problem import SimpleProblem
from primal_dual import PrimalDualTrainer

import torch

torch.set_default_dtype(torch.float64)




In [29]:
class OriginalQPProblem(SimpleProblem):
    def __init__(self, Q, p, A, G, b, d, valid_frac=0.1, test_frac=0.1):
        super().__init__(Q, p, A, G, b, d, valid_frac, test_frac)

        self._X = self._b
        self._num = self._X.shape[0]
        self._neq = self._A.shape[0]
        self._nineq = self._G.shape[0]
        self._xdim = self._X.shape[1]

    def eq_resid(self, X, Y):
        return X - Y @ self.A.T

    def ineq_resid(self, X, Y):
        return Y @ self.G.T - self.d

    
    def opt_solve(self, X, solver_type="osqp", tol=1e-4):
        if solver_type == "osqp":
            print("running osqp")
            Q, p, A, G, d = self.Q_np, self.p_np, self.A_np, self.G_np, self.d_np
            X_np = X.detach().cpu().numpy()
            Y = []
            total_time = 0
            for Xi in X_np:
                solver = osqp.OSQP()
                my_A = np.vstack([A, G])
                my_l = np.hstack([Xi, -np.ones(d.shape[0]) * np.inf])
                my_u = np.hstack([Xi, d])
                solver.setup(
                    P=csc_matrix(Q),
                    q=p,
                    A=csc_matrix(my_A),
                    l=my_l,
                    u=my_u,
                    verbose=False,
                    eps_prim_inf=tol,
                )
                start_time = time.time()
                results = solver.solve()
                end_time = time.time()

                total_time += end_time - start_time
                if results.info.status == "solved":
                    Y.append(results.x)
                else:
                    Y.append(np.ones(self.ydim) * np.nan)

                sols = np.array(Y)
                parallel_time = total_time / len(X_np)
        else:
            raise NotImplementedError

        return sols, total_time, parallel_time

    def calc_Y(self):
        Y = self.opt_solve(self.X)[0]
        feas_mask = ~np.isnan(Y).all(axis=1)
        self._num = feas_mask.sum()
        self._X = self._X[feas_mask]
        self._Y = torch.tensor(Y[feas_mask])
        return Y

class QPProblemVaryingG(SimpleProblem):
    def __init__(self, Q, p, A, G_base, G_varying, b, d, n_varying_rows, valid_frac=0.1, test_frac=0.1):
        super().__init__(Q, p, A, G_varying, b, d, valid_frac, test_frac)
        self.G_base = torch.tensor(G_base)
        self.n_varying_rows = n_varying_rows
        # Take the first n rows of G as input
        self._X = self._G[:, :n_varying_rows, :].flatten(start_dim=1)
        self._num = self._X.shape[0]
        self._neq = self._A.shape[0]
        # G now has num_samples in first dimension, num_constraints in second dimension. Take second dimension!
        self._nineq = self._G.shape[1]
        self._xdim = self._X.shape[1]

    def eq_resid(self, X, Y):
        """RHS of equality constraints now remains constant across problem instances."""
        return self.b - Y @ self.A.T

    def rebuild_G_from_X(self, X):
        # Reshape X to match the first self.n_varying_rows rows of G
        custom_G = X.reshape(X.shape[0], self.n_varying_rows, self._ydim)  # Reshape for the batch size

        # Take only the first sample of G and clone it for modification
        G = self.G_base.clone()  # Shape is (M, P)

        # Repeat G for the batch size to avoid memory overlap
        G = G.unsqueeze(0).repeat(X.shape[0], 1, 1)  # Shape is (batch_size, M, P)

        # Assign custom_G to the first self.n_varying_rows rows of G
        G[:, :self.n_varying_rows, :] = custom_G  # Ensure dimensions match
        return G
    
    def ineq_resid(self, X, Y):
        """
        For the ineq resid, we need to extract the first n rows of the G matrix from it's flattened form X, and plug them into G.
        """

        G = self.rebuild_G_from_X(X)

        # resid = Y @ G.transpose(1, 2) - h
        residual = torch.bmm(Y.unsqueeze(1), G.transpose(1, 2)).squeeze(1) - self.d

        # Compute inequality residual
        return residual
    
    def opt_solve(self, X, solver_type="osqp", tol=1e-4):
        """We change op_solve so that the varying G matrices are taken from the input X.
        """
        if solver_type == "osqp":
            print("running osqp")
            Q, p, b, d = self.Q_np, self.p_np, self.b_np, self.d_np
            G_np = self.rebuild_G_from_X(X).detach().cpu().numpy()
            A = self.A_np
            Y = []
            total_time = 0
            for Gi in G_np:
                solver = osqp.OSQP()
                my_A = np.vstack([A, Gi])
                my_l = np.hstack([b, -np.ones(d.shape[0]) * np.inf])
                my_u = np.hstack([b, d])
                solver.setup(
                    P=csc_matrix(Q),
                    q=p,
                    A=csc_matrix(my_A),
                    l=my_l,
                    u=my_u,
                    verbose=False,
                    eps_prim_inf=tol,
                )
                start_time = time.time()
                results = solver.solve()
                end_time = time.time()

                total_time += end_time - start_time
                if results.info.status == "solved":
                    Y.append(results.x)
                else:
                    Y.append(np.ones(self.ydim) * np.nan)

            sols = np.array(Y)
            parallel_time = total_time / len(X)

        else:
            raise NotImplementedError

        return sols, total_time, parallel_time

    def calc_Y(self):
        Y = self.opt_solve(self.X)[0]
        feas_mask = ~np.isnan(Y).all(axis=1)
        self._num = feas_mask.sum()
        self._X = self._X[feas_mask]
        self._Y = torch.tensor(Y[feas_mask])
        return Y

class QPProblemVaryingGbd(SimpleProblem):
    def __init__(self, Q, p, A, G_base, G_varying, b, d, n_varying_rows, valid_frac=0.1, test_frac=0.1):
        super().__init__(Q, p, A, G_varying, b, d, valid_frac, test_frac)
        self.G_base = torch.tensor(G_base)
        self.n_varying_rows = n_varying_rows
        # Flatten the rows of G that are varying, to be added to the NN input.
        G_flattened = self.G[:, :n_varying_rows, :].flatten(start_dim=1)
        self._X = torch.concat([G_flattened, self.b, self.d], dim=1)
        self._num = self._X.shape[0]
        self._neq = self._A.shape[0]
        # G now has num_samples in first dimension, num_constraints in second dimension. Take second dimension!
        self._nineq = self._G.shape[1]
        self._xdim = self._X.shape[1]

    def eq_resid(self, X, Y):
        """B is now varying, we should extract it from X"""
        G, b, d = self.rebuild_Gbd_from_X(X)
        return b - Y @ self.A.T

    def rebuild_Gbd_from_X(self, X):
        # Reshape X to match the first self.n_varying_rows rows of G
        G_size = self.n_varying_rows*self.ydim
        b_size = self.neq
        flattened_G = X[:, :G_size]
        b = X[:, G_size:G_size+b_size]
        d = X[:, G_size+b_size:]
        custom_G = flattened_G.reshape(X.shape[0], self.n_varying_rows, self._ydim)  # Reshape for the batch size

        # Take only the first sample of G and clone it for modification
        G = self.G_base.clone()  # Shape is (M, P)

        # Repeat G for the batch size to avoid memory overlap
        G = G.unsqueeze(0).repeat(X.shape[0], 1, 1)  # Shape is (batch_size, M, P)

        # Assign custom_G to the first self.n_varying_rows rows of G
        G[:, :self.n_varying_rows, :] = custom_G  # Ensure dimensions match
        return G, b, d
    
    def ineq_resid(self, X, Y):
        """
        For the ineq resid, we need to extract the first n rows of the G matrix from it's flattened form X, and plug them into G.
        """

        G, b, d = self.rebuild_Gbd_from_X(X)

        # resid = Y @ G.transpose(1, 2) - h
        residual = torch.bmm(Y.unsqueeze(1), G.transpose(1, 2)).squeeze(1) - d

        # Compute inequality residual
        return residual
    
    def opt_solve(self, X, solver_type="osqp", tol=1e-4):
        """We change op_solve so that the varying G matrices are taken from the input X.
        """
        if solver_type == "osqp":
            print("running osqp")
            Q, p, b, d = self.Q_np, self.p_np, self.b_np, self.d_np
            G, b, d = self.rebuild_Gbd_from_X(X)
            G_np, b_np, d_np = G.detach().cpu().numpy(), b.detach().cpu().numpy(), d.detach().cpu().numpy()
            A = self.A_np
            Y = []
            total_time = 0
            for idx, Gi in enumerate(G_np):
                solver = osqp.OSQP()
                my_A = np.vstack([A, Gi])
                my_l = np.hstack([b_np[idx], -np.ones(d_np[idx].shape[0]) * np.inf])
                my_u = np.hstack([b_np[idx], d_np[idx]])
                solver.setup(
                    P=csc_matrix(Q),
                    q=p,
                    A=csc_matrix(my_A),
                    l=my_l,
                    u=my_u,
                    verbose=False,
                    eps_prim_inf=tol,
                )
                start_time = time.time()
                results = solver.solve()
                end_time = time.time()

                total_time += end_time - start_time
                if results.info.status == "solved":
                    Y.append(results.x)
                else:
                    Y.append(np.ones(self.ydim) * np.nan)

            sols = np.array(Y)
            parallel_time = total_time / len(X)

        else:
            raise NotImplementedError

        return sols, total_time, parallel_time

    def calc_Y(self):
        Y = self.opt_solve(self.X)[0]
        feas_mask = ~np.isnan(Y).all(axis=1)
        self._num = feas_mask.sum()
        self._X = self._X[feas_mask]
        self._Y = torch.tensor(Y[feas_mask])
        return Y
    

class ScaledLPProblem(SimpleProblem):
    def __init__(self, Q, p, A, G, b, d, obj_coeff, valid_frac=0.1, test_frac=0.1):
        super().__init__(Q, p, A, G, b, d, valid_frac, test_frac)

        self._X = self._b
        self._c = torch.tensor(obj_coeff)
        self._num = self._X.shape[0]
        self._neq = self._A.shape[0]
        self._nineq = self._G.shape[0]
        self._xdim = self._X.shape[1]

    def eq_resid(self, X, Y):
        return X - Y @ self.A.T

    def ineq_resid(self, X, Y):
        return Y @ self.G.T - self.d

    def obj_fn(self, Y):
        return Y @ self._c.T

    
    def opt_solve(self, X, solver_type="osqp", tol=1e-4):
        if solver_type == "osqp":
            print("running osqp")
            Q, p, A, G, d = self.Q_np, self.p_np, self.A_np, self.G_np, self.d_np
            c = self._c.numpy()
            X_np = X.detach().cpu().numpy()
            Y = []
            total_time = 0
            zero_Q = np.zeros((c.shape[0], c.shape[0]))

            for Xi in X_np:
                solver = osqp.OSQP()
                my_A = np.vstack([A, G])
                my_l = np.hstack([Xi, -np.ones(d.shape[0]) * np.inf])
                my_u = np.hstack([Xi, d])
                solver.setup(
                    q=c,
                    A=csc_matrix(my_A),
                    l=my_l,
                    u=my_u,
                    verbose=False,
                    eps_prim_inf=tol,
                )
                start_time = time.time()
                results = solver.solve()
                end_time = time.time()

                total_time += end_time - start_time
                if results.info.status == "solved":
                    Y.append(results.x)
                else:
                    Y.append(np.ones(self.ydim) * np.nan)

                sols = np.array(Y)
                parallel_time = total_time / len(X_np)
        else:
            raise NotImplementedError

        return sols, total_time, parallel_time

    def calc_Y(self):
        Y = self.opt_solve(self.X)[0]
        feas_mask = ~np.isnan(Y).all(axis=1)
        self._num = feas_mask.sum()
        self._X = self._X[feas_mask]
        self._Y = torch.tensor(Y[feas_mask])
        return Y

In [37]:
def create_QP_dataset(num_var, num_ineq, num_eq, num_examples):
    np.random.seed(17)
    Q = np.diag(np.random.random(num_var))
    p = np.random.random(num_var)
    A = np.random.normal(loc=0, scale=1., size=(num_eq, num_var))
    X = np.random.uniform(-1, 1, size=(num_examples, num_eq))
    G = np.random.normal(loc=0, scale=1., size=(num_ineq, num_var))
    h = np.sum(np.abs(G@np.linalg.pinv(A)), axis=1)

    problem = OriginalQPProblem(Q, p, A, G, X, h)
    problem.calc_Y()
    print(len(problem.Y))

    with open("./QP_data/original/random_simple_dataset_var{}_ineq{}_eq{}_ex{}".format(num_var, num_ineq, num_eq, num_examples), 'wb') as f:
        pickle.dump(problem, f)
    
    return problem

def create_varying_G_dataset(num_var, num_ineq, num_eq, num_examples, num_varying_rows):
    """Creates a modified QP data set that differs in the inequality constraint matrix, instead of the RHS variables.
    """
    np.random.seed(17)
    Q = np.diag(np.random.random(num_var))
    p = np.random.random(num_var)
    A = np.random.normal(loc=0, scale=1., size=(num_eq, num_var))
    # X is the same for all samples:
    b = np.random.uniform(-1, 1, size=(num_eq))
    G_base = np.random.normal(loc=0, scale=1., size=(num_ineq, num_var))
    # TODO: Can we keep h constant, if we are varying G?
    d = np.sum(np.abs(G_base@np.linalg.pinv(A)), axis=1)

    G_list = []
    # For each sample, create a different inequality constraint matrix
    for _ in range(num_examples):
        G_sample = G_base.copy()
        # Vary the first n rows, (specified by num_varying_rows).
        G_sample[:num_varying_rows, :] = np.random.normal(loc=0, scale=1., size=(1, num_var))
        G_list.append(G_sample)

    G = np.array(G_list)
    problem = QPProblemVaryingG(Q=Q, p=p, A=A, G_base=G_base, G_varying=G, b=b, d=d, n_varying_rows=num_varying_rows)
    problem.calc_Y()
    print(len(problem.Y))

    with open("./QP_data/modified/MODIFIED_random_simple_dataset_var{}_ineq{}_eq{}_ex{}".format(num_var, num_ineq, num_eq, num_examples), 'wb') as f:
        pickle.dump(problem, f)
    
    return problem

def create_varying_G_b_d_dataset(num_var, num_ineq, num_eq, num_examples, num_varying_rows):
    """Creates a modified QP data set that differs in the inequality constraint matrix, instead of the RHS variables.
    """
    np.random.seed(17)
    Q = np.diag(np.random.random(num_var))
    p = np.random.random(num_var)
    A = np.random.normal(loc=0, scale=1., size=(num_eq, num_var))
    # X is the same for all samples:
    B = np.random.uniform(-1, 1, size=(num_examples, num_eq))
    G_base = np.random.normal(loc=0, scale=1., size=(num_ineq, num_var))

    G_list = []
    # For each sample, create a different inequality constraint matrix
    for _ in range(num_examples):
        G_sample = G_base.copy()
        # Vary the first n rows, (specified by num_varying_rows).
        G_sample[:num_varying_rows, :] = np.random.normal(loc=0, scale=1., size=(num_varying_rows, num_var))
        G_list.append(G_sample)
    
    # Create H matrix for each example
    D_list = []
    for Gi in G_list:
        d = np.sum(np.abs(Gi @ np.linalg.pinv(A)), axis=1)  # Compute bounds for all inequalities
        D_list.append(d)  # Resulting shape will be (num_ineq,)

    G = np.array(G_list)
    D = np.stack(D_list, axis=0)  # Shape (num_examples, num_ineq)
    problem = QPProblemVaryingGbd(Q=Q, p=p, A=A, G_base=G_base, G_varying=G, b=B, d=D, n_varying_rows=num_varying_rows)
    problem.calc_Y()
    print(len(problem.Y))

    with open("./QP_data/modified/MODIFIED_random_simple_dataset_var{}_ineq{}_eq{}_ex{}".format(num_var, num_ineq, num_eq, num_examples), 'wb') as f:
        pickle.dump(problem, f)
    
    return problem

def create_LP_dataset(num_var, num_ineq, num_eq, num_examples, scale='normal'):
    if scale == 'normal':
        obj_scale = 1
        var_scale = 1
        rhs_scale = 1
    elif scale == 'large':
        obj_scale = 1e9
        var_scale = 1e3
        rhs_scale = 1e3

    np.random.seed(17)
    c = np.random.uniform(-obj_scale, obj_scale, size=num_var)

    Q = np.diag(np.random.random(num_var))
    p = np.random.random(num_var)
    A = np.random.normal(loc=0, scale=var_scale, size=(num_eq, num_var))
    X = np.random.uniform(-rhs_scale, rhs_scale, size=(num_examples, num_eq))
    G = np.random.normal(loc=0, scale=var_scale, size=(num_ineq, num_var))
    h = np.sum(np.abs(G@np.linalg.pinv(A)), axis=1)

    problem = ScaledLPProblem(Q, p, A, G, X, h, c)
    problem.calc_Y()
    print(len(problem.Y))

    # with open("./QP_data/original/random_simple_dataset_var{}_ineq{}_eq{}_ex{}".format(num_var, num_ineq, num_eq, num_examples), 'wb') as f:
        # pickle.dump(problem, f)
    
    return problem

def create_scaled_QP_problem(num_var, num_ineq, num_eq, num_examples, scale='normal'):
    if scale == 'normal':
        obj_scale = 1
        var_scale = 1
        rhs_scale = 1
    elif scale == 'large':
        obj_scale = 1e9
        var_scale = 1e3
        rhs_scale = 1e3

    np.random.seed(17)

    Q = np.diag(np.random.random(num_var)) * obj_scale
    p = np.random.random(num_var)
    A = np.random.normal(loc=0, scale=var_scale, size=(num_eq, num_var))
    X = np.random.uniform(-rhs_scale, rhs_scale, size=(num_examples, num_eq))
    G = np.random.normal(loc=0, scale=var_scale, size=(num_ineq, num_var))
    h = np.sum(np.abs(G@np.linalg.pinv(A)), axis=1)

    problem = OriginalQPProblem(Q, p, A, G, X, h)
    problem.calc_Y()
    print(len(problem.Y))

    # with open("./QP_data/original/random_simple_dataset_var{}_ineq{}_eq{}_ex{}".format(num_var, num_ineq, num_eq, num_examples), 'wb') as f:
        # pickle.dump(problem, f)
    
    return problem

In [39]:
# normal scale LP test!
num_var = 100
num_ineq = 50
num_eq = 50
num_examples = 100
SCALE = 'large'

problem = create_scaled_QP_problem(num_var=num_var, num_ineq=num_ineq, num_eq=num_eq, num_examples=num_examples, scale=SCALE)

print(problem.obj_fn(problem.Y).mean().item())

DEVICE = 'cpu'
args = {
        "K": 10,
        "L": 500,
        "tau": 0.8,
        "rho": 0.5,
        "rho_max": 5000,
        "alpha": 10,
        "batch_size": 200,
        "hidden_sizes": [500, 500],
        "primal_lr": 1e-4,
        "dual_lr": 1e-4,
        "decay": 0.99,
        "patience": 10,
        "corrEps": 1e-4,
}

run_name = "test"
save_dir = os.path.join('outputs', 'QP_experiments',
    run_name + "-" + str(time.time()).replace('.', '-'))

if not os.path.exists(save_dir):
    os.makedirs(save_dir)
with open(os.path.join(save_dir, 'args.dict'), 'wb') as f:
    pickle.dump(args, f)

# Run PDL
trainer = PrimalDualTrainer(problem, args, save_dir, problem_type="Benchmark", log=True)
original_primal_net, original_dual_net, original_stats = trainer.train_PDL()

running osqp
100
tensor(1.4042e+08)
X dim: 50
Y dim: 100
Size of mu: 50
Size of lambda: 50
Epoch 0 done. Time taken: 6.242254018783569. Rho: 0.5. Primal LR: 0.0001, Dual LR: 8.429431933839271e-05
0: p-loss: 3.0885E+15, obj. val 2.8630E+15, Max eq.: 9.1313E+06, Max ineq.: 7.2171E+06, Mean eq.: 2.7898E+06, Mean ineq.: 1.3286E+06

Epoch 1 done. Time taken: 6.377027988433838. Rho: 0.5. Primal LR: 9.605960100000001e-05, Dual LR: 5.309055429551135e-05
1: p-loss: 3.0539E+15, obj. val 2.8299E+15, Max eq.: 9.0544E+06, Max ineq.: 7.1770E+06, Mean eq.: 2.7642E+06, Mean ineq.: 1.3424E+06

Epoch 2 done. Time taken: 6.636106967926025. Rho: 0.5. Primal LR: 6.825545950103872e-05, Dual LR: 3.377544008989021e-05
2: p-loss: 3.0476E+15, obj. val 2.8230E+15, Max eq.: 9.0396E+06, Max ineq.: 7.1687E+06, Mean eq.: 2.7645E+06, Mean ineq.: 1.3532E+06

Epoch 3 done. Time taken: 6.408543109893799. Rho: 0.5. Primal LR: 4.475232137638109e-05, Dual LR: 2.1272570322901873e-05
3: p-loss: 3.0465E+15, obj. val 2.8217E+1

In [19]:
num_var = 100
num_ineq = 50
num_eq = 50
num_examples = 1
original_problem = create_QP_dataset(num_var=num_var, num_ineq=num_ineq, num_eq=num_eq, num_examples=num_examples)
print(original_problem.Y.shape)
print(original_problem.G.shape)

DEVICE = 'cpu'
args = {
        "K": 10,
        "L": 500,
        "tau": 0.8,
        "rho": 0.5,
        "rho_max": 5000,
        "alpha": 10,
        "batch_size": 200,
        "hidden_sizes": [500, 500],
        "primal_lr": 1e-4,
        "dual_lr": 1e-4,
        "decay": 0.99,
        "patience": 10,
        "corrEps": 1e-4,
}

run_name = "test"
save_dir = os.path.join('outputs', 'QP_experiments',
    run_name + "-" + str(time.time()).replace('.', '-'))

if not os.path.exists(save_dir):
    os.makedirs(save_dir)
with open(os.path.join(save_dir, 'args.dict'), 'wb') as f:
    pickle.dump(args, f)

# Run PDL
trainer = PrimalDualTrainer(original_problem, args, save_dir, problem_type="Benchmark", log=True)
original_primal_net, original_dual_net, original_stats = trainer.train_PDL()

running osqp
1
torch.Size([1, 100])
torch.Size([50, 100])
X dim: 50
Y dim: 100
Size of mu: 50
Size of lambda: 50


ValueError: batch_size should be a positive integer value, but got batch_size=0

In [7]:
num_var = 10
num_ineq = 5
num_eq = 5
num_examples = 100
num_varying_rows = 5
modified_problem = create_varying_G_dataset(num_var=num_var, num_ineq=num_ineq, num_eq=num_eq, num_examples=num_examples, num_varying_rows=num_varying_rows)

DEVICE = 'cpu'
args = {
        "K": 10,
        "L": 500,
        "tau": 0.8,
        "rho": 0.5,
        "rho_max": 5000,
        "alpha": 10,
        "batch_size": 200,
        "hidden_sizes": [500, 500],
        "primal_lr": 1e-4,
        "dual_lr": 1e-4,
        "decay": 0.99,
        "patience": 10,
        "corrEps": 1e-4,
}

run_name = "test"
save_dir = os.path.join('outputs', 'QP_experiments',
    run_name + "-" + str(time.time()).replace('.', '-'))

if not os.path.exists(save_dir):
    os.makedirs(save_dir)
with open(os.path.join(save_dir, 'args.dict'), 'wb') as f:
    pickle.dump(args, f)

# Run PDL
trainer = PrimalDualTrainer(modified_problem, args, save_dir, problem_type="Benchmark", log=True)
modified_primal_net, modified_dual_net, modified_stats = trainer.train_PDL()

running osqp
100
X dim: 50
Y dim: 10
Size of mu: 5
Size of lambda: 5
Epoch 0 done. Time taken: 7.658443212509155. Rho: 0.5. Primal LR: 7.249803359578537e-05, Dual LR: 6.491026283684025e-05
0: p-loss: 2.4716E+00, obj. val -3.9875E+00, Max eq.: 2.8629E+00, Max ineq.: 5.7728E-01, Mean eq.: 1.3132E+00, Mean ineq.: 4.4015E-01

Epoch 1 done. Time taken: 7.1610798835754395. Rho: 0.5. Primal LR: 4.6122196741809574e-05, Dual LR: 4.1294967113388845e-05
1: p-loss: 1.9140E+00, obj. val -2.7137E+00, Max eq.: 2.4776E+00, Max ineq.: 5.0813E-01, Mean eq.: 1.2119E+00, Mean ineq.: 3.7762E-01

Epoch 2 done. Time taken: 7.317880868911743. Rho: 0.5. Primal LR: 2.9048849430996377e-05, Dual LR: 2.6008546137772605e-05
2: p-loss: 1.9665E+00, obj. val -2.5046E+00, Max eq.: 2.4716E+00, Max ineq.: 5.0474E-01, Mean eq.: 1.2028E+00, Mean ineq.: 3.7423E-01

Epoch 3 done. Time taken: 7.222394227981567. Rho: 0.5. Primal LR: 1.848045639485463e-05, Dual LR: 1.6546259566473476e-05
3: p-loss: 2.0111E+00, obj. val -2.4902E

In [ ]:
opt_Y = torch.tensor(original_problem.calc_Y())
opt_obj_values = original_problem.obj_fn(opt_Y)
predicted_Y = original_primal_net(original_problem.trainX)
predicted_obj_values = original_problem.obj_fn(predicted_Y)

print(f"Original problem -- opt: {opt_obj_values.mean()}, pred: {predicted_obj_values.mean()}")


opt_obj_values = modified_problem.obj_fn(torch.tensor(modified_problem.Y[:80]))
predicted_Y = modified_primal_net(modified_problem.trainX)
predicted_obj_values = original_problem.obj_fn(predicted_Y)

print(f"Modified problem -- opt: {opt_obj_values.mean()}, pred: {predicted_obj_values.mean()}")

running osqp
Original problem -- opt: -18.500195218984732, pred: -18.450353886636613
Modified problem -- opt: -17.31410062843396, pred: -17.119726465504737
torch.Size([80, 50])
torch.Size([80, 1000])


/var/folders/yn/jbnxb2792bx_bmgd675jx1fh0000gq/T/ipykernel_83511/3526701426.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  opt_obj_values = modified_problem.obj_fn(torch.tensor(modified_problem.Y[:80]))


In [42]:
num_var = 50
num_ineq = 50
num_eq = 50
num_examples = 100
num_varying_rows = 10
modified_problem = create_varying_G_b_d_dataset(num_var=num_var, num_ineq=num_ineq, num_eq=num_eq, num_examples=num_examples, num_varying_rows=num_varying_rows)

DEVICE = 'cpu'
args = {
        "K": 10,
        "L": 500,
        "tau": 0.8,
        "rho": 0.5,
        "rho_max": 5000,
        "alpha": 10,
        "batch_size": 200,
        "hidden_sizes": [500, 500],
        "primal_lr": 1e-4,
        "dual_lr": 1e-4,
        "decay": 0.99,
        "patience": 10,
        "corrEps": 1e-4,
}

run_name = "test"
save_dir = os.path.join('outputs', 'QP_experiments',
    run_name + "-" + str(time.time()).replace('.', '-'))

if not os.path.exists(save_dir):
    os.makedirs(save_dir)
with open(os.path.join(save_dir, 'args.dict'), 'wb') as f:
    pickle.dump(args, f)

# Run PDL
trainer = PrimalDualTrainer(modified_problem, args, save_dir, problem_type="Benchmark", log=True)
modified_primal_net, modified_dual_net, modified_stats = trainer.train_PDL()

running osqp
100
X dim: 600
Y dim: 50
Size of mu: 50
Size of lambda: 50


KeyboardInterrupt: 

In [27]:
opt_obj_values = modified_problem.obj_fn(torch.tensor(modified_problem.Y[:80]))
predicted_obj_values = modified_problem.obj_fn(modified_primal_net(modified_problem.trainX))

print(f"Modified problem -- opt: {opt_obj_values.mean()}, pred: {predicted_obj_values.mean()}")

Modified problem -- opt: -17.933158508477426, pred: -17.543311041351224


/var/folders/yn/jbnxb2792bx_bmgd675jx1fh0000gq/T/ipykernel_21687/1671212256.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  opt_obj_values = modified_problem.obj_fn(torch.tensor(modified_problem.Y[:80]))
